In [1]:
import tensorflow as tf
#import numpy as np
Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 6   # Polynmial order
Δt = T / m  # interval between two exercise dates

# Construct polynomial features of order up to k using the
# recursive formulation
def chebyshev_basis(x, k):  
    B = [tf.ones(len(x)), x]   
    for n in range(2, k):
        Bn = 2 * x * B[n - 1] - B[n - 2] 
        B.append(Bn)       

    return tf.stack(B,axis = 1)

# scales x to be in the interval(-1, 1)
def scale(x):
    xmin = tf.reduce_min(x)
    xmax = tf.reduce_max(x)
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b

# simulates the stock price evolution
def advance(S, r, σ, Δt, n):
    dB = tf.sqrt(Δt) * tf.random.normal(shape=tf.shape(S))
    out = S + r * S * Δt + σ * S * dB
    return out

# LSMC algorithm

def compute_price(order, Spot, σ, K, r):
    tf.random.set_seed(0)
    S0 = Spot * tf.ones(n)
    S = {0: S0}

    for t in range(m):
        S[t + 1] = advance(S[t], r, σ, Δt, n)

    discount = tf.math.exp(-r * Δt)
    CFL = {t: tf.maximum(0., K - S[t]) for t in S.keys()}
    value_tp1 = CFL[m] * discount 
    CV = {m: tf.zeros_like(S0)}

    for t in range(m - 1, 0, -1):
        X = chebyshev_basis(scale(S[t]), order) 
        Y = tf.constant(value_tp1, shape=(n, 1)) 

        # regression to estimate the continuation value
        Θ = tf.linalg.solve(tf.matmul(tf.transpose(X),X),tf.matmul(tf.transpose(X),Y))    
        CV[t] = tf.constant(tf.matmul(X,Θ),dtype=tf.float32)
        CFL[t] = tf.constant(CFL[t],shape=(n,1),dtype=tf.float32)
        exercise = tf.math.greater(CFL[t],CV[t])
        value_tp1 = tf.constant(value_tp1, shape=(n,1))
        value_t = tf.where(exercise, CFL[t], value_tp1)
        value_tp1 = discount * value_t
        

    POF = {t: tf.where(CV[t] < CFL[t], CFL[t], 0) for t in range(1, m + 1)}
    POF[m] = tf.constant(POF[m],shape=(n,1))
    POF[m-1] = tf.constant(POF[m-1],shape=(n,1))
    #print(POF.values())  

    POF = tf.stack(list(POF.values()),axis=0)

    POF = tf.constant(POF, shape=(m,n))
    idx_payoffs = tf.math.argmax(POF > 0, axis=0)
    FPOF = tf.transpose(tf.one_hot(idx_payoffs, m)) * POF
    m_range = tf.constant(range(0, m), shape=(m,1),dtype=tf.float32)
    dFPOF = FPOF * tf.math.exp(-r * m_range * Δt)
    dFPOF=tf.reduce_sum(dFPOF)
    PRICE = dFPOF / n
    return(PRICE)

print(compute_price(order, Spot, σ, K, r))

tf.Tensor(4.487445, shape=(), dtype=float32)
